# Lab 2: Building Creative AI Agents with LangChain
**Generative AI and Computational Creativity**

## Learning Objectives
By the end of this lab, you will:
1. Understand the architecture of AI agents vs. simple chatbots
2. Build custom tools for creative tasks
3. Create a ReAct agent using LangChain and HuggingFace models
4. Apply agents to computational creativity tasks

**Duration**: 80 minutes

---

## Part 1: Conceptual Introduction (5 minutes)

### What Makes an Agent?

**Traditional Chatbot/LLM**:
- User prompt → LLM → Response
- Single-step reasoning
- No external tool use

**AI Agent**:
- Observes → Thinks → Acts → Observes (iterative)
- Can use external tools (calculators, APIs, custom functions)
- Plans multi-step solutions
- Has memory of past actions

### Agent Components
1. **Brain (LLM)**: Reasoning engine
2. **Tools**: External capabilities (search, calculate, generate, etc.)
3. **Memory**: Short-term (conversation) and long-term (facts/experiences)
4. **Planning**: Ability to break down complex tasks

### ReAct Pattern (Reasoning + Acting)
```
Thought: I need to find rhyming words for "creative"
Action: Use rhyme_finder tool with input "creative"
Observation: Found rhymes: native,rative, etc.
Thought: Now I can construct a poem
Final Answer: [Generated poem]
```

---

## Part 2: Environment Setup (10 minutes)

### Model Selection Strategy (15GB GPU RAM)

We'll use **task-specific models** rather than one large model:

1. **Agent Brain**: `google/flan-t5-large` (780M params, ~3GB VRAM)
   - Excellent at instruction following and reasoning
   - Fast inference for agent decision-making
   
2. **Creative Generation**: `EleutherAI/pythia-1.4b` (1.4B params, ~3GB VRAM)
   - Good for creative text generation
   - Or use `meta-llama/Llama-3.2-1B-Instruct` if available

3. **Specialized Tasks**: Load on-demand
   - Sentiment: `distilbert-base-uncased-finetuned-sst-2-english` (~250MB)
   - Embeddings: `sentence-transformers/all-MiniLM-L6-v2` (~80MB)

**Total**: ~6-7GB, leaving room for activations and batching

In [ ]:
# Install required packages
!pip install -q langchain langchain-huggingface transformers accelerate bitsandbytes pronouncing nltk sentencepiece
!pip install -q langchain-community

In [ ]:
import torch
import nltk
import pronouncing
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)

print("✓ Packages imported successfully")
print(f"✓ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

### Load the Agent Brain Model (Flan-T5-Large)

In [ ]:
# Load Flan-T5-Large for agent reasoning
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# Create a pipeline
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    top_p=0.95
)

# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print("✓ Agent brain model loaded (Flan-T5-Large)")

# Test it
test_response = llm.invoke("What is computational creativity?")
print(f"\nTest response: {test_response}")

---
## Part 3: Building Custom Tools (20 minutes)

Tools are Python functions that agents can call. Each tool needs:
1. A descriptive name
2. A clear description (helps agent decide when to use it)
3. A function that takes a string input and returns a string output

### Tool 1: Rhyme Finder
Finds rhyming words for poetry generation

In [ ]:
def find_rhymes(word: str) -> str:
    """
    Finds rhyming words for a given word using the pronouncing library.
    """
    word = word.strip().lower()
    rhymes = pronouncing.rhymes(word)
    
    if not rhymes:
        return f"No rhymes found for '{word}'"
    
    # Return top 10 rhymes
    top_rhymes = rhymes[:10]
    return f"Rhymes for '{word}': {', '.join(top_rhymes)}"

# Test the function
print(find_rhymes("creative"))
print(find_rhymes("AI"))

### Tool 2: Syllable Counter
Counts syllables for meter and rhythm in poetry

In [ ]:
def count_syllables(text: str) -> str:
    """
    Counts syllables in a word or phrase.
    """
    text = text.strip().lower()
    words = text.split()
    
    total_syllables = 0
    word_counts = []
    
    for word in words:
        # Remove punctuation
        word = ''.join(c for c in word if c.isalnum())
        phones = pronouncing.phones_for_word(word)
        
        if phones:
            syllable_count = pronouncing.syllable_count(phones[0])
            total_syllables += syllable_count
            word_counts.append(f"{word}({syllable_count})")
        else:
            # Fallback: simple vowel counting
            syllable_count = len([c for c in word if c in 'aeiou'])
            total_syllables += max(1, syllable_count)
            word_counts.append(f"{word}(~{max(1, syllable_count)})")
    
    return f"Total syllables: {total_syllables}. Breakdown: {' '.join(word_counts)}"

# Test
print(count_syllables("computational creativity"))
print(count_syllables("AI agents are fascinating"))

### Tool 3: Style Analyzer
Analyzes the sentiment and emotional tone of text

In [ ]:
# Load a lightweight sentiment model
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=0 if torch.cuda.is_available() else -1
)

def analyze_style(text: str) -> str:
    """
    Analyzes the emotional tone and style of text.
    """
    if len(text.strip()) < 3:
        return "Text too short to analyze."
    
    # Get sentiment
    sentiment = sentiment_analyzer(text[:512])[0]  # Limit to 512 chars
    
    # Basic statistics
    words = text.split()
    avg_word_length = sum(len(w) for w in words) / len(words) if words else 0
    sentence_count = text.count('.') + text.count('!') + text.count('?')
    
    analysis = f"""Style Analysis:
- Sentiment: {sentiment['label']} (confidence: {sentiment['score']:.2f})
- Word count: {len(words)}
- Avg word length: {avg_word_length:.1f} characters
- Sentences: {max(1, sentence_count)}
- Tone: {'Formal' if avg_word_length > 5 else 'Casual'}"""
    
    return analysis

# Test
print(analyze_style("I absolutely love creating beautiful poetry with AI! It's amazing!"))
print("\n" + "="*50 + "\n")
print(analyze_style("The dark storm clouds gathered ominously over the desolate landscape."))

### Tool 4: Metaphor Generator
Creates metaphors by finding conceptual connections

In [ ]:
# Load a small creative text generation model
# We'll use this selectively to avoid memory issues
metaphor_tokenizer = AutoTokenizer.from_pretrained("gpt2")
metaphor_model = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    device_map="auto",
    torch_dtype=torch.float16
)

metaphor_pipe = pipeline(
    "text-generation",
    model=metaphor_model,
    tokenizer=metaphor_tokenizer,
    max_new_tokens=50,
    temperature=0.9,
    do_sample=True,
    pad_token_id=metaphor_tokenizer.eos_token_id
)

def generate_metaphor(concept: str) -> str:
    """
    Generates creative metaphors for a given concept.
    """
    prompt = f"Complete this creative metaphor: {concept.strip()} is like"
    
    try:
        result = metaphor_pipe(prompt, num_return_sequences=1)[0]['generated_text']
        # Extract just the metaphor part
        metaphor = result.replace(prompt, "").strip()
        # Clean up - take first sentence
        metaphor = metaphor.split('.')[0] + '.' if '.' in metaphor else metaphor
        return f"Metaphor for '{concept}': {concept} is like{metaphor}"
    except Exception as e:
        return f"Could not generate metaphor: {str(e)}"

# Test
print(generate_metaphor("artificial intelligence"))
print(generate_metaphor("creativity"))

### 🎯 EXERCISE 1: Create Your Own Tool (5 minutes)

Create a tool that would be useful for creative writing. Ideas:
- Word association finder
- Alliteration checker
- Character name generator
- Plot twist suggester

Your tool should:
1. Take a string input
2. Return a string output
3. Have a clear, specific purpose

In [ ]:
# YOUR CODE HERE
def my_creative_tool(input_text: str) -> str:
    """
    TODO: Implement your creative tool here.
    
    Example ideas:
    - Find words that start with the same letter (alliteration)
    - Generate character traits based on a name
    - Suggest plot complications
    - Create sensory descriptions (visual, auditory, etc.)
    """
    pass

# Test your tool
# print(my_creative_tool("test input"))

---
## Part 4: Creating the Agent (15 minutes)

Now we'll combine our tools with the LLM to create a ReAct agent.

### Step 1: Define Tools for LangChain

In [ ]:
# Wrap our functions as LangChain Tools
tools = [
    Tool(
        name="RhymeFinder",
        func=find_rhymes,
        description="Useful for finding words that rhyme with a given word. Input should be a single word. Returns a list of rhyming words."
    ),
    Tool(
        name="SyllableCounter",
        func=count_syllables,
        description="Useful for counting syllables in words or phrases to maintain poetic meter. Input should be a word or phrase. Returns syllable count."
    ),
    Tool(
        name="StyleAnalyzer",
        func=analyze_style,
        description="Analyzes the emotional tone, sentiment, and style of text. Input should be a sentence or paragraph. Returns analysis of tone, formality, and sentiment."
    ),
    Tool(
        name="MetaphorGenerator",
        func=generate_metaphor,
        description="Generates creative metaphors for concepts. Input should be a concept or idea. Returns a creative metaphor."
    ),
]

# If you created a custom tool, add it here:
# tools.append(Tool(
#     name="MyTool",
#     func=my_creative_tool,
#     description="Description of what your tool does..."
# ))

print(f"✓ Loaded {len(tools)} tools for the agent")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")

### Step 2: Create the ReAct Agent Prompt

The prompt template guides the agent's reasoning process.

In [ ]:
# ReAct prompt template
react_prompt = PromptTemplate.from_template(
    """You are a creative writing assistant AI agent. You help with poetry, storytelling, and creative text generation.

You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""
)

print("✓ Agent prompt template created")

### Step 3: Initialize the Agent

In [ ]:
# Create the ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_prompt
)

# Create an executor to run the agent
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # This will show us the agent's thinking process
    handle_parsing_errors=True,
    max_iterations=5  # Prevent infinite loops
)

print("✓ Creative Writing Agent initialized and ready!")

---
## Part 5: Testing the Agent (10 minutes)

Let's test our agent with various creative tasks!

### Test 1: Simple Tool Usage

In [ ]:
# Test: Find rhymes
response = agent_executor.invoke({
    "input": "Find me 5 words that rhyme with 'night' for my poem."
})

print("\n" + "="*50)
print("FINAL ANSWER:")
print(response['output'])

### Test 2: Multi-Step Reasoning

In [ ]:
# Test: Create a haiku (requires syllable counting)
response = agent_executor.invoke({
    "input": """Help me check if this haiku has the correct syllable structure (5-7-5):
    
    Artificial minds
    Creating words and pictures  
    New art emerges
    """
})

print("\n" + "="*50)
print("FINAL ANSWER:")
print(response['output'])

### Test 3: Creative Generation Task

In [ ]:
# Test: Generate a metaphor and analyze its style
response = agent_executor.invoke({
    "input": "Create a metaphor about 'machine learning' and then analyze its emotional tone."
})

print("\n" + "="*50)
print("FINAL ANSWER:")
print(response['output'])

### 🎯 EXERCISE 2: Test Your Own Prompts (5 minutes)

Try asking the agent to:
1. Help you write a rhyming couplet
2. Analyze the tone of a piece of text you provide
3. Create a metaphor and find rhyming words for it

Observe how the agent:
- Decides which tools to use
- Chains multiple tool calls together
- Reasons about the task

In [ ]:
# YOUR EXPERIMENTS HERE
my_prompt = """# TODO: Write your creative prompt here
"""

# response = agent_executor.invoke({"input": my_prompt})
# print("\n" + "="*50)
# print("FINAL ANSWER:")
# print(response['output'])

---
## Part 6: Advanced Agent - Creative Story Assistant (20 minutes)

Let's build a more sophisticated agent for collaborative storytelling.

### Additional Tools for Story Writing

In [ ]:
def character_consistency_check(description: str) -> str:
    """
    Checks if character descriptions maintain consistency.
    Uses simple keyword matching for traits.
    """
    # Define trait keywords
    trait_categories = {
        'personality': ['brave', 'cowardly', 'kind', 'cruel', 'smart', 'foolish', 'confident', 'shy'],
        'appearance': ['tall', 'short', 'young', 'old', 'strong', 'weak', 'beautiful', 'plain'],
        'emotional': ['happy', 'sad', 'angry', 'calm', 'excited', 'bored', 'anxious', 'relaxed']
    }
    
    description_lower = description.lower()
    found_traits = {}
    
    for category, traits in trait_categories.items():
        found_traits[category] = [trait for trait in traits if trait in description_lower]
    
    # Check for contradictions
    contradictions = []
    if 'brave' in description_lower and 'cowardly' in description_lower:
        contradictions.append("brave vs. cowardly")
    if 'tall' in description_lower and 'short' in description_lower:
        contradictions.append("tall vs. short")
    if 'kind' in description_lower and 'cruel' in description_lower:
        contradictions.append("kind vs. cruel")
    
    result = "Character Consistency Analysis:\n"
    for category, traits in found_traits.items():
        if traits:
            result += f"- {category.capitalize()}: {', '.join(traits)}\n"
    
    if contradictions:
        result += f"\n⚠️  Potential contradictions: {', '.join(contradictions)}"
    else:
        result += "\n✓ No obvious contradictions detected"
    
    return result

# Test
test_desc = "A brave young warrior, tall and confident, but also cowardly when facing dragons."
print(character_consistency_check(test_desc))

In [ ]:
def plot_suggester(genre: str) -> str:
    """
    Suggests plot elements based on genre.
    """
    plot_elements = {
        'fantasy': [
            "A magical artifact is discovered in an unlikely place",
            "The protagonist learns they have hidden magical abilities",
            "An ancient prophecy begins to unfold",
            "A forbidden realm becomes accessible"
        ],
        'scifi': [
            "First contact with an alien civilization occurs",
            "A time paradox threatens reality",
            "AI gains unexpected consciousness",
            "A breakthrough in technology has unforeseen consequences"
        ],
        'mystery': [
            "A trusted ally's true motives are revealed",
            "An impossible crime is committed",
            "Hidden connections between characters emerge",
            "Evidence points to an unexpected culprit"
        ],
        'romance': [
            "A misunderstanding drives the lovers apart",
            "Circumstances force them to work together",
            "A secret from the past resurfaces",
            "External pressures test their relationship"
        ]
    }
    
    genre = genre.strip().lower()
    
    if genre in plot_elements:
        import random
        suggestions = random.sample(plot_elements[genre], min(3, len(plot_elements[genre])))
        return f"Plot suggestions for {genre}:\n" + "\n".join(f"- {s}" for s in suggestions)
    else:
        available = ', '.join(plot_elements.keys())
        return f"Genre '{genre}' not recognized. Available genres: {available}"

# Test
print(plot_suggester("fantasy"))
print("\n")
print(plot_suggester("scifi"))

### Create Enhanced Story Writing Agent

In [ ]:
# Enhanced tool set
story_tools = tools + [
    Tool(
        name="CharacterConsistency",
        func=character_consistency_check,
        description="Checks character descriptions for consistency and contradictions. Input should be a character description. Returns analysis of traits and any contradictions."
    ),
    Tool(
        name="PlotSuggester",
        func=plot_suggester,
        description="Suggests plot developments based on genre. Input should be a genre (fantasy, scifi, mystery, or romance). Returns plot suggestions."
    ),
]

# Create story agent
story_agent = create_react_agent(
    llm=llm,
    tools=story_tools,
    prompt=react_prompt
)

story_agent_executor = AgentExecutor(
    agent=story_agent,
    tools=story_tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=6
)

print(f"✓ Story Writing Agent created with {len(story_tools)} tools")

### Test Complex Story Task

In [ ]:
story_prompt = """
I'm writing a fantasy story. Help me:
1. Suggest an interesting plot twist
2. Check this character description for consistency: 
   "Elena is a brave knight who fears nothing, though she trembles at the sight of magic."
3. Find words that rhyme with 'sword' for a battle poem I want to include
"""

response = story_agent_executor.invoke({"input": story_prompt})

print("\n" + "="*50)
print("FINAL ANSWER:")
print(response['output'])

### 🎯 EXERCISE 3: Build a Multi-Step Creative Task (10 minutes)

Design a complex prompt that requires the agent to:
1. Use at least 3 different tools
2. Make decisions based on tool outputs
3. Create something creative (poem, story opening, dialogue, etc.)

Example ideas:
- "Write a 4-line poem about AI with perfect AABB rhyme scheme and check its tone"
- "Create a character, check for consistency, suggest a plot based on their traits"
- "Generate metaphors for 'data' and 'algorithm', then use them in a haiku"

In [ ]:
# YOUR COMPLEX CREATIVE TASK HERE
complex_prompt = """
# TODO: Design your multi-step creative task
# Think about:
# - What creative output do you want?
# - Which tools would help achieve it?
# - How should the agent combine tool outputs?
"""

# response = story_agent_executor.invoke({"input": complex_prompt})
# print("\n" + "="*50)
# print("FINAL ANSWER:")
# print(response['output'])

---
## Part 7: Understanding Agent Limitations (5 minutes)

### Common Challenges with Agents

1. **Parsing Errors**: The LLM might not always follow the exact format
2. **Hallucination**: May try to use tools that don't exist
3. **Infinite Loops**: Can get stuck without proper max_iterations
4. **Tool Selection**: May not always pick the optimal tool
5. **Context Limits**: Long conversations can exceed model context window

### Debugging Tips

In [ ]:
# Example: Deliberately problematic prompt
try:
    response = agent_executor.invoke({
        "input": "Use the StoryGenerator tool to write me a story",
        "max_iterations": 3  # Limit iterations for safety
    })
    print(response['output'])
except Exception as e:
    print(f"Error occurred: {type(e).__name__}")
    print(f"Message: {str(e)}")
    print("\nThis is expected! The 'StoryGenerator' tool doesn't exist.")
    print("The agent should gracefully handle this with handle_parsing_errors=True")

---
## Part 8: Advanced Topics & Extensions (5 minutes)

### Multi-Agent Systems
You could create multiple specialized agents:
- **Writer Agent**: Generates creative content
- **Editor Agent**: Reviews and suggests improvements
- **Fact-Checker Agent**: Verifies consistency

### Memory Integration
From last week's lab on memory-aware systems:
```python
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")
agent_with_memory = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,  # Remembers past interactions
    verbose=True
)
```

### Custom Agent Types
- **Plan-and-Execute**: Plans entire sequence before acting
- **Tree-of-Thought**: Explores multiple reasoning paths
- **Reflexion**: Self-reflects and improves over iterations

---
## Homework Assignment

### Task: Extend the Creative Agent

**Choose ONE of the following:**

1. **Add 2 New Creative Tools**
   - Examples: Dialogue generator, Setting describer, Character name generator, Conflict suggester
   - Test them with complex multi-step prompts
   
2. **Integrate Memory**
   - Add ConversationBufferMemory to your agent
   - Create a multi-turn conversation where the agent remembers context
   - Example: Iteratively develop a character or story

3. **Build a Specialized Creative Agent**
   - Choose a specific creative domain (e.g., poetry, screenwriting, game narratives)
   - Create 3-4 domain-specific tools
   - Design a custom prompt template for that domain

**Deliverable**: 
- Modified notebook with your implementation
- 2-3 example interactions demonstrating your additions
- Brief writeup (200-300 words) on:
  - What you built and why
  - Challenges you encountered
  - Potential improvements

**Due**: Next lab session

---
## Additional Resources

### Documentation
- [LangChain Agents Documentation](https://python.langchain.com/docs/modules/agents/)
- [HuggingFace Transformers](https://huggingface.co/docs/transformers/)
- [ReAct Paper](https://arxiv.org/abs/2210.03629)

### Interesting Models to Try
- **Reasoning**: `google/flan-t5-xl`, `meta-llama/Llama-3.2-3B-Instruct`
- **Creative Writing**: `EleutherAI/pythia-2.8b`, `gpt2-large`
- **Specialized**: `google/flan-t5-base` fine-tuned on specific tasks

### Papers & Articles
- ReAct: Synergizing Reasoning and Acting in Language Models
- Generative Agents: Interactive Simulacra of Human Behavior
- Constitutional AI: Harmlessness from AI Feedback

---
## Summary

### What We Learned
✅ Difference between chatbots and agents  
✅ Building custom tools for creative tasks  
✅ Creating ReAct agents with LangChain  
✅ Integrating HuggingFace models efficiently  
✅ Debugging and understanding agent limitations  

### Key Takeaways
1. **Agents = LLM + Tools + Planning**: They can break down complex tasks
2. **Tool Quality Matters**: Well-designed tools enable better agent performance
3. **Verbose Mode**: Essential for understanding agent reasoning
4. **Model Selection**: Choose models based on task and resource constraints
5. **Iteration Limits**: Prevent infinite loops and manage costs

### Next Steps
- Experiment with different agent architectures
- Try multi-agent systems
- Explore domain-specific applications
- Integrate with external APIs and databases

---

**Questions? Discussion? Experiments?**